In [ ]:
import numpy as np
import math
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
import seaborn as sns

A)Data1

In [ ]:
#reading data points 1
pts = np.zeros((1,2))
i=0
with open(r'./SpectData.txt') as file:
    for readline in file:
        line_strip = readline.strip() # removing \n
        point = line_strip.split('\t') # removing \t

        if(i==0):
            pts[0]=point
        else:
            pts=np.append(pts,np.array([point]),axis=0)  
        i+=1             
data=pts.astype(float)

In [ ]:
k=KMeans(n_clusters=2).fit(data)
k=k.predict(data)

sns.scatterplot(x=data[:,0], y=data[:,1],hue=k)
plt.savefig("Kmeans.jpg")

Labels

In [ ]:
def euclid_distance(p1,p2):
    return math.sqrt(((p2[0]-p1[0])**2)+((p2[1]-p1[1])**2))

avg_x=np.average(data[:,0:1])
avg_y=np.average(data[:,1:2])

avg=np.array([avg_x,avg_y])
print(avg)
distances=np.zeros(2084)
for i in range(2084):
    distances[i]=euclid_distance(data[i],avg)

labels=np.zeros(2084)
for i in range(2084):
    if(distances[i]>0.6):
        labels[i]=0
    else:
        labels[i]=1

sns.scatterplot(x=data[:,0], y=data[:,1],hue=labels)        

In [ ]:
def weightFunc(pts,sigma):
    res=np.zeros((len(pts),len(pts)))
    for i in range(len(pts)):
        for j in range(len(pts)):
            if(i != j):
                calc=math.exp((-1*(math.sqrt(((pts[i][0]-pts[j][0])**2)+((pts[i][1]-pts[j][1])**2)))**2)/(2*(sigma**2)))
                res[i][j] = calc         
    return(res)


In [ ]:
sigma = np.arange(0.01, 1.01, 0.01).tolist()
W = []
for i in range(len(sigma)):
    W.append(weightFunc(data,sigma[i]))

In [ ]:
def getD(weight):
    D = np.zeros((len(weight),len(weight)))
    for i in range(len(weight)):
        count = 0
        for j in range(len(weight)):
            count += weight[i][j]
        D[i][i] = count 
    return D

In [ ]:
D = []
for i in range(len(sigma)):
    D.append(getD(W[i]))

In [ ]:
W_numpy = np.array(W)
D_numpy = np.array(D)
subtracted_array = np.subtract(D_numpy, W_numpy)
L = list(subtracted_array)

In [ ]:
def eigen_vectors(Lap): #L: Laplacian
    arr_of_eig_vectors=[]
    for i in range(100):
        eig_values,eig_vectors=np.linalg.eigh(Lap[i])
        arr_of_eig_vectors.append(eig_vectors[:,1:2])#eigh sorts eigenvalues 

    return arr_of_eig_vectors

arr_of_eig_vectors=eigen_vectors(L)

#proof that eigen values are always sorted when eigh is used
# y=np.zeros(100)
# for i in range(100):
#     eig_values,eig_vectors=np.linalg.eigh(L[i])
#     eig_values_sorted=eig_values
#     eig_values_sorted=np.sort(eig_values_sorted)
#     if((eig_values_sorted==eig_values).all()==True):
#         y[i]=1
# print(y.all()==1)



In [ ]:
def calcPredictions(eig_vectors):
    predictions=np.zeros((100,2084))
    for i in range(100):
        for j in range(2084):
            if(eig_vectors[i][j]>0):
                predictions[i][j]=1
            else:
                predictions[i][j]=0
    return predictions

predictions=calcPredictions(arr_of_eig_vectors)


In [ ]:
def calcAccuracies(p,l):
    accuracies=np.zeros(100)
    for i in range(100):
        count=0
        for j in range(2084):
            if(p[i][j]==l[j]):
                count+=1
        if(count==0):
            count=2084

        accuracies[i]=count/2084*100

    return accuracies

accuracies=calcAccuracies(predictions,labels)
    
max_accuracy=np.max(accuracies)    
sigma_of_max_accuracy=np.where(accuracies==np.max(accuracies))[0][0]

print("Maximim Accuracy = "+str(max_accuracy))
print("using sigma = "+str((sigma_of_max_accuracy+1)/100))

sns.scatterplot(x=data[:,0], y=data[:,1],hue=predictions[sigma_of_max_accuracy]) #plot of the best accuracy
plt.savefig("clustering_result_a.jpg")

In [ ]:
sns.scatterplot(data=accuracies) #plot of the clustering accuracy versus different values of the standard deviation
plt.savefig("Spect_a.jpg")

Another Approach

In [ ]:
def predict(L):
    predictions=np.zeros((100,2084))

    for i in range(len(sigma)):
        eig_values,eig_vectors=np.linalg.eigh(L[i])
        U=eig_vectors[:,1:2]
        kmean=KMeans(n_clusters=2).fit(U.reshape(-1,1))
        predictions[i]=kmean.predict(U.reshape(-1,1))

    return predictions
    
another_predictions=predict(L) 

In [ ]:
another_accuracies=calcAccuracies(another_predictions,labels)
    
max_accuracy=np.max(another_accuracies)    
sigma_of_max_accuracy=0.01+np.where(accuracies==np.max(accuracies))[0][0]

print("Maximim Accuracy = "+str(max_accuracy))
print("using sigma = "+str((sigma_of_max_accuracy+1)/100))
print(another_accuracies)


In [ ]:
sns.scatterplot(x=data[:,0], y=data[:,1],hue=another_predictions[sigma_of_max_accuracy])
plt.savefig("Kmeans_a_another_approach.jpg")


In [ ]:
sns.scatterplot(data=another_accuracies) #plot of the clustering accuracy versus different values of the standard deviation
plt.savefig("Spect_a_another_approach.jpg")

B)Data2

In [ ]:
#reading data points 2
pts = np.zeros((1,2))
i=0
with open(r'./SpectData2.txt') as file:
    for readline in file:
        line_strip = readline.strip() # removing \n
        point = line_strip.split('\t') # removing \t

        if(i==0):
            pts[0]=point
        else:
            pts=np.append(pts,np.array([point]),axis=0)  
        i+=1             
data2=pts.astype(float)


In [ ]:
avg_x2=np.average(data2[:,0:1])
avg_y2=np.average(data2[:,1:2])

avg2=np.array([avg_x2,avg_y2])
print(avg2)
distances2=np.zeros(2084)
for i in range(2084):
    distances2[i]=euclid_distance(data2[i],avg2)

labels2=np.zeros(2084)
for i in range(2084):
    if(distances2[i]>0.6):
        labels2[i]=0
    else:
        labels2[i]=1

sns.scatterplot(x=data2[:,0], y=data2[:,1],hue=labels2)        

In [ ]:
W = []
for i in range(len(sigma)):
    W.append(weightFunc(data2,sigma[i]))

In [ ]:
D = []
for i in range(len(sigma)):
    D.append(getD(W[i]))

In [ ]:
W_numpy = np.array(W)
D_numpy = np.array(D)
subtracted_array = np.subtract(D_numpy, W_numpy)
L = list(subtracted_array)


In [ ]:
arr_of_eig_vectors2=eigen_vectors(L)

In [ ]:
predictions2 = calcPredictions(arr_of_eig_vectors2)

In [ ]:
accuracies2=calcAccuracies(predictions2,labels2)
    
max_accuracy2=np.max(accuracies2)    
sigma_of_max_accuracy2=np.where(accuracies2==np.max(accuracies2))[0][0]

print("Maximim Accuracy = "+str(max_accuracy2))
print("using sigma = "+str((sigma_of_max_accuracy2+1)/100))

sns.scatterplot(x=data2[:,0], y=data2[:,1],hue=predictions2[sigma_of_max_accuracy2]) #plot of the best accuracy
plt.savefig("clustering_result_b.jpg")

In [ ]:
sns.scatterplot(data=accuracies2) #plot of the clustering accuracy versus different values of the standard deviation
plt.savefig("Spect_b.jpg")

C)

In [ ]:
small_radius=euclid_distance(data[3],avg)
large_radius=euclid_distance(data[0],avg)

small_radius2=euclid_distance(data2[0],avg2)
large_radius2=euclid_distance(data2[1],avg2)

print("difference1 = "+str(large_radius-small_radius)+" sigma = "+str((sigma_of_max_accuracy+1)/100))
print("difference2 = "+str(large_radius2-small_radius2)+" sigma = "+str((sigma_of_max_accuracy2+1)/100))

print("\nwe found that when the difference between the two radii of the concentric circle increases, the value of optimium standard deviation increases.")



